This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/pick.html).  I recommend having both windows open, side-by-side!

In [1]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    JacobianWrtVariable,
    JointSliders,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    StartMeshcat,
)

from manipulation import ConfigureParser, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Kinematic Jacobians for the UR5e manipulator

This example computes the geometric Jacobian matrix J_G(q) that relates joint velocities to the TCP spatial velocity.

The Jacobian is a 6×6 matrix (6 rows for spatial velocity: 3 angular + 3 linear, 6 columns for UR5e joints).

Move the joint sliders to see how the Jacobian changes with robot configuration. The smallest singular value indicates proximity to singularities.

In [ ]:
class PrintJacobian(LeafSystem):
    def __init__(self, plant, frame):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._frame = frame
        self.DeclareVectorInputPort("state", plant.num_multibody_states())
        self.DeclareForcedPublishEvent(self.Publish)

    def Publish(self, context):
        state = self.get_input_port().Eval(context)
        self._plant.SetPositionsAndVelocities(self._plant_context, state)
        W = self._plant.world_frame()
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._frame,
            [0, 0, 0],
            W,
            W,
        )  ## This is the important line

        print("J_G:")
        print(np.array2string(J_G, formatter={"float": lambda x: "{:5.1f}".format(x)}))
        print(
            f"smallest singular value(J_G): {np.min(np.linalg.svd(J_G, compute_uv=False))}"
        )
        clear_output(wait=True)


def ur5e_jacobian_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    # Register UR5e package
    parser.package_map().Add('ur5e_description', '/home/ajay/drake/ur5e_description')
    # Load composite model (creates 'ur5e' and 'netft_probe' model instances)
    parser.AddModels('/home/ajay/drake/ur5e_description/ur5e_netft_probe.dmd.yaml')
    plant.Finalize()

    # Optional: Add coordinate frame triads
    ur5e_frames = [
        "base_link_inertia", "shoulder_link", "upper_arm_link", "forearm_link",
        "wrist_1_link", "wrist_2_link", "wrist_3_link", "flange"
    ]
    for frame_name in ur5e_frames:
        AddMultibodyTriad(plant.GetFrameByName(frame_name), scene_graph)

    netft_probe_frames = ["netft_link", "p42v_link", "tcp"]
    for frame_name in netft_probe_frames:
        AddMultibodyTriad(plant.GetFrameByName(frame_name), scene_graph)

    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )

    # Get TCP frame
    G = plant.GetFrameByName("tcp")
    print_jacobian = builder.AddSystem(PrintJacobian(plant, G))
    builder.Connect(plant.get_state_output_port(), print_jacobian.get_input_port())

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


meshcat.DeleteAddedControls()
ur5e_jacobian_example()

INFO:drake:Press the 'Stop JointSliders' button in Meshcat or press 'Escape' to continue.


J_G:
[[  0.0   0.0   0.0   0.0  -0.0   0.0]
 [  0.0   1.0   1.0   1.0  -0.0   1.0]
 [  1.0  -0.0  -0.0  -0.0  -1.0  -0.0]
 [ -0.5  -0.1  -0.1  -0.1   0.4   0.0]
 [  0.8  -0.0  -0.0   0.0  -0.0   0.0]
 [  0.0  -0.8  -0.4   0.0  -0.0   0.0]]
smallest singular value(J_G): 9.517460276492759e-17


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05e7ca3d-5650-42c9-9342-60bc45e7d545' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>